In [ ]:
import numpy as np
import pandas as pd

np.random.seed(42)

def generate_quake_geophys_samples(label, n, center):

    return pd.DataFrame({
        "seismic_moment_rate":        np.random.normal(center["moment_rate"], 5e15, n),
        "surface_displacement_rate":  np.random.normal(center["displacement"], 10, n),
        "coulomb_stress_change":      np.random.normal(center["stress"], 100, n),
        "average_focal_depth":        np.random.normal(center["depth"], 8, n),
        "fault_slip_rate":            np.random.normal(center["slip_rate"], 2, n),
        "quake_binary":               label
    })

active_subduction = generate_quake_geophys_samples(1, 1200, {
    "moment_rate": 2e16, "displacement": 50, "stress": 600,
    "depth": 30, "slip_rate": 12
})

slow_deforming_plate = generate_quake_geophys_samples(0, 800, {
    "moment_rate": 1e15, "displacement": 5, "stress": 50,
    "depth": 10, "slip_rate": 2
})

post_quake_aftershock_zone = generate_quake_geophys_samples(0, 500, {
    "moment_rate": 3e15, "displacement": 20, "stress": 200,
    "depth": 15, "slip_rate": 1.5
})

stress_transfer_zone = generate_quake_geophys_samples(1, 700, {
    "moment_rate": 1.5e16, "displacement": 35, "stress": 480,
    "depth": 25, "slip_rate": 10
})

tectonic_noise = generate_quake_geophys_samples(1, 400, {
    "moment_rate": 6e15, "displacement": 18, "stress": 320,
    "depth": 12, "slip_rate": 6
})

# --- Final Assembly ---
data = pd.concat([
    active_subduction,
    slow_deforming_plate,
    post_quake_aftershock_zone,
    stress_transfer_zone,
    tectonic_noise
])

data = data.clip(lower=0).sample(frac=1, random_state=42).reset_index(drop=True)
data.to_csv("dataset/earthquake_data.csv", index=False)

print("QuakeNet Dataset built:", data.shape)

QuakeNet Dataset built: (3600, 6)


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.saving import register_keras_serializable
import pandas as pd
from sklearn.model_selection import train_test_split
import tf2onnx

data = pd.read_csv("dataset/earthquake_data.csv")
X = data.drop("quake_binary", axis=1).astype("float32")
y = data["quake_binary"].astype("float32")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.3, random_state=42
)

@register_keras_serializable()
class StressAmplifier(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        stress = inputs[:, 2]
        slip = inputs[:, 4]
        stress_boost = tf.sigmoid((stress - 400) * 0.01)
        slip_boost = tf.sigmoid((slip - 8) * 0.5)
        modulation = 1.0 + 0.4 * stress_boost * slip_boost
        return tf.expand_dims(modulation, axis=-1)

@register_keras_serializable()
class DepthSuppressor(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        depth = inputs[:, 3]
        suppression = tf.sigmoid((depth - 25) * 0.15)
        modulation = 1.0 - 0.3 * suppression
        return tf.expand_dims(modulation, axis=-1)

@register_keras_serializable()
class DisplacementActivator(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def call(self, inputs):
        displacement = inputs[:, 1]
        activation = tf.sigmoid((displacement - 30) * 0.08)
        modulation = 1.0 + 0.3 * activation
        return tf.expand_dims(modulation, axis=-1)

@register_keras_serializable()
class SoftScale(tf.keras.layers.Layer):
    def __init__(self, factor=0.25, **kwargs):
        super().__init__(**kwargs)
        self.factor = factor

    def call(self, inputs):
        return 1.0 + self.factor * tf.tanh(inputs - 1.0)

input_layer = tf.keras.Input(shape=(5,), name="quake_inputs")

x = tf.keras.layers.BatchNormalization()(input_layer)
x1 = tf.keras.layers.Dense(128, activation="relu")(x)
x2 = tf.keras.layers.Dense(64, activation="relu")(x1)
x3 = tf.keras.layers.Dense(64, activation="relu")(x2)
residual = tf.keras.layers.Add()([x3, x2])
raw_logits = tf.keras.layers.Dense(1)(residual)

mod_strength = tf.keras.layers.Multiply()([
    StressAmplifier()(input_layer),
    DepthSuppressor()(input_layer),
    DisplacementActivator()(input_layer)
])
mod_scaled = SoftScale()(mod_strength)
mod_logits = tf.keras.layers.Dense(1)(mod_scaled)

logits = tf.keras.layers.Add()([raw_logits, mod_logits])
output = tf.keras.layers.Activation("sigmoid")(logits)

model = tf.keras.Model(inputs=input_layer, outputs=output)
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

early_stop = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
model.fit(X_train, y_train, validation_split=0.2, epochs=15, batch_size=16, callbacks=[early_stop])
model.summary()
loss, acc = model.evaluate(X_test, y_test)
print(f"QuakeNet Accuracy: {acc:.4f}")
model.save("QuakeNet2.h5")

model_proto, _ = tf2onnx.convert.from_keras(model)
with open(f"models/ONNX/QuakeNet.onnx", "wb") as f:
    f.write(model_proto.SerializeToString())

Epoch 1/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8290 - loss: 0.3253 - val_accuracy: 0.9127 - val_loss: 0.2190
Epoch 2/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9541 - loss: 0.1203 - val_accuracy: 0.9583 - val_loss: 0.1204
Epoch 3/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9471 - loss: 0.1215 - val_accuracy: 0.9504 - val_loss: 0.1142
Epoch 4/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9571 - loss: 0.1184 - val_accuracy: 0.9504 - val_loss: 0.1189
Epoch 5/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9595 - loss: 0.1124 - val_accuracy: 0.9504 - val_loss: 0.1155
Epoch 6/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9571 - loss: 0.1031 - val_accuracy: 0.9524 - val_loss: 0.1141
Epoch 7/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9486 - loss: 0.1444 - val_accuracy: 0.9504 - val_loss: 0.1137
Epoch 8/15
126/126 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9498 - loss: 0.1316 - val_accuracy: 0.

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ quake_inputs        │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 5)         │         20 │ quake_inputs[0][… │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 128)       │        768 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      8,256 │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stress_amplifier    │ (None, 1)         │          0 │ quake_inputs[0][… │
│ (StressAmplifier)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depth_suppressor    │ (None, 1)         │          0 │ quake_inputs[0][… │
│ (DepthSuppressor)   │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ displacement_activ… │ (None, 1)         │          0 │ quake_inputs[0][… │
│ (DisplacementActiv… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64)        │      4,160 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply (Multiply) │ (None, 1)         │          0 │ stress_amplifier… │
│                     │                   │            │ depth_suppressor… │
│                     │                   │            │ displacement_act… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 64)        │          0 │ dense_9[0][0],    │
│                     │                   │            │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ soft_scale          │ (None, 1)         │          0 │ multiply[0][0]    │
│ (SoftScale)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 1)         │         65 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 1)         │          2 │ soft_scale[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 1)         │          0 │ dense_10[0][0],   │
│                     │                   │            │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_1        │ (None, 1)         │          0 │ add_2[0][0]       │
│ (Activation)        │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,795 (155.45 KB)

 Trainable params: 13,261 (51.80 KB)

 Non-trainable params: 10 (40.00 B)

 Optimizer params: 26,524 (103.61 KB)

34/34 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9704 - loss: 0.0890


QuakeNet Accuracy: 0.9759


ERROR:tf2onnx.tfonnx:rewriter <function rewrite_constant_fold at 0x0000024584186B60>: exception `np.cast` was removed in the NumPy 2.0 release. Use `np.asarray(arr, dtype=dtype)` instead.
